In [80]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

In [81]:
# Read csv file in data folder
filename = os.listdir(path='data/transactions')[0]

print(filename)

CIBC_240819.csv


In [82]:
# Create dataframe
cols = ['TRANSACTION_DATE','DESCRIPTION_ONE','P1','P2','CARD']

df = pd.read_csv(f'data/transactions/{filename}', header=None, names=cols)

print(df.head())

  TRANSACTION_DATE                     DESCRIPTION_ONE     P1     P2  \
0       2024-08-15  ROGERS ******0235 888-764-3771, ON  56.50    NaN   
1       2024-08-07   PAYMENT THANK YOU/PAIEMEN T MERCI    NaN  23.06   
2       2024-08-06     APPLE.COM/BILL 866-712-7753, ON   2.81    NaN   
3       2024-08-06     APPLE.COM/BILL 866-712-7753, ON   1.46    NaN   
4       2024-08-02        Spotify P2E30B83BC Stockholm   6.77    NaN   

               CARD  
0  4505********3187  
1  4505********3187  
2  4505********3187  
3  4505********3187  
4  4505********3187  


In [83]:
# Combine value columns
df['CAD'] = np.where(df['P2'].notna(), df['P2'], 0) - \
            np.where(df['P1'].notna(), df['P1'], 0)

# Drop P1, P2, and CARD columns
df.drop(columns=['P1','P2','CARD'], inplace=True)

# Remove credit card payments
df.drop(df[df['DESCRIPTION_ONE'] == 'PAYMENT THANK YOU/PAIEMEN T MERCI'].index, inplace=True)

print(df.head())

  TRANSACTION_DATE                     DESCRIPTION_ONE    CAD
0       2024-08-15  ROGERS ******0235 888-764-3771, ON -56.50
2       2024-08-06     APPLE.COM/BILL 866-712-7753, ON  -2.81
3       2024-08-06     APPLE.COM/BILL 866-712-7753, ON  -1.46
4       2024-08-02        Spotify P2E30B83BC Stockholm  -6.77
5       2024-07-26             PRESTO MOBL TORONTO, ON -16.29


In [84]:
# Create a new column 'TRANSACTION_TYPE'
df['TRANSACTION_TYPE'] = np.where(df['CAD'] > 0, 'Income', 'Expense')

# Create a new column 'TRANSACTION_YM'
df['TRANSACTION_YM'] = df['TRANSACTION_DATE'].str[0:7]

print(df.head())

  TRANSACTION_DATE                     DESCRIPTION_ONE    CAD  \
0       2024-08-15  ROGERS ******0235 888-764-3771, ON -56.50   
2       2024-08-06     APPLE.COM/BILL 866-712-7753, ON  -2.81   
3       2024-08-06     APPLE.COM/BILL 866-712-7753, ON  -1.46   
4       2024-08-02        Spotify P2E30B83BC Stockholm  -6.77   
5       2024-07-26             PRESTO MOBL TORONTO, ON -16.29   

  TRANSACTION_TYPE TRANSACTION_YM  
0          Expense        2024-08  
2          Expense        2024-08  
3          Expense        2024-08  
4          Expense        2024-08  
5          Expense        2024-07  


In [85]:
# Function to replace based on keywords
def replace_based_on_keywords(value):
    if 'APPLE' in value:
        return 'APPLE iCloud & Google Photos'
    elif 'ROGERS' in value:
        return 'ROGERS Phone Bill'
    elif 'Spotify' in value:
        return 'Spotify'
    elif 'PRESTO' in value:
        return 'PRESTO'
    return value 

df['DESCRIPTION_ONE'] = df['DESCRIPTION_ONE'].apply(replace_based_on_keywords)

print(df.head())

In [86]:
load_dotenv('secrets.env')

# SQL Connection
uid = 'sa'
pwd = os.getenv('SQL_SERVER_PWD')
server = os.getenv('DATABASE_URL')
database = 'FINANCIAL_TRANSACTIONS'
driver = 'ODBC+Driver+17+for+SQL+Server'

SQL_conn_str = f'mssql+pyodbc://{uid}:{pwd}@{server}/{database}?driver={driver}'
sql_conn = create_engine(SQL_conn_str)

In [87]:
# Empty Data
empty_query = \
text("""
TRUNCATE TABLE [dbo].[CIBC];
""")

cursor = sql_conn.connect()
cursor.execute(empty_query)
cursor.commit()
cursor.close()

In [88]:
# Export
df.to_excel('Data Validation/transaction_data_cibc.xlsx', index=False)
df.to_sql('CIBC', sql_conn, schema='dbo', if_exists='append', index=False)

288

In [89]:
# Close SQL connection
sql_conn.dispose()